In [1]:
import pandas as pd
import time
import boto3

In [2]:
# 'rootkey.csv' is the file you've downloaded from AWS
with open('rootkey.csv', 'r') as f:
    content = f.readlines()

keys = {}
for line in content:
    pair = line.strip().split('=')
    keys.update({pair[0] : pair[1]})

AWS_ACCESS_KEY_ID = keys['AWSAccessKeyId']
AWS_SECRET_KEY = keys['AWSSecretKey']

In [6]:
def amazon_transcribe(filename):

    transcribe = boto3.client('transcribe', 
                              region_name='us-east-2',
                              aws_access_key_id=AWS_ACCESS_KEY_ID,
                              aws_secret_access_key=AWS_SECRET_KEY)
    
    job_name = "naep-job-{}".format(filename)
    job_uri = "s3://naep-vocab/{}".format(filename)
    transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': job_uri},
        MediaFormat='wav',
        LanguageCode='en-US',
        #OutputBucketName='string',
        Settings={
            'VocabularyName': 'naep-cust-voc'
        }
    )

    while True:
        result = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        if result['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        time.sleep(15)
    if result['TranscriptionJob']['TranscriptionJobStatus'] == "COMPLETED":
        data = pd.read_json(result['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    return data['results'][1][0]['transcript']

In [7]:
ORIG_DIR = './audio/orig'
import os

In [8]:
res = []
for filename in os.listdir(ORIG_DIR):
    try:
        transcript = amazon_transcribe(filename)
        res.append((filename, transcript))
    except:
        print('Boo', filename)

Boo Score3_168692.wav


In [9]:
with open('./aws_results_vocab.txt', 'w') as f:
    for pair in res:
        f.write(pair[0])
        f.write('\n')
        f.write(pair[1])
        f.write('\n\n\n')